In [28]:
import json
import time, datetime
from pymongo import MongoClient
import pandas as pd
import os
from dateutil import parser

### Convert mongo json into csv

In [2]:
db_url = 'ec2-54-67-97-244.us-west-1.compute.amazonaws.com:27017'
db_name = 'activities'

In [3]:
# start_day = datetime.datetime(2015, 1, 1)

In [4]:
mongocli = MongoClient(db_url)# init mongodb client
mongodb = mongocli[db_name] # connect 'github' database

In [5]:
stars = mongodb['repos.IanLunn.Hover.stargazers'].find({"starred_at": {"$gte": '2015-01-01'}},
                                                        {"_id":0, "starred_at": 1, "user.type": 1, "user.id":1, "user.login":1, "user.site_admin":1})
stars.count(with_limit_and_skip=True) # enable limit

9592

In [6]:
for s in stars.limit(10):
    print ['IanLunn', 'Hover'] + [s['starred_at']] + s['user'].values()

['IanLunn', 'Hover', u'2015-01-01T09:41:31Z', u'jason-gu', False, u'User', 2045134]
['IanLunn', 'Hover', u'2015-01-02T07:28:13Z', u'ddliu', False, u'User', 797146]
['IanLunn', 'Hover', u'2015-01-02T16:48:40Z', u'gopalindians', False, u'User', 1937433]
['IanLunn', 'Hover', u'2015-01-02T23:21:49Z', u'Siot', False, u'User', 4410120]
['IanLunn', 'Hover', u'2015-01-03T08:48:12Z', u'cxwe', False, u'User', 9068824]
['IanLunn', 'Hover', u'2015-01-03T11:42:45Z', u'putaoshu', False, u'User', 406587]
['IanLunn', 'Hover', u'2015-01-03T13:28:07Z', u'eLmre', False, u'User', 3106214]
['IanLunn', 'Hover', u'2015-01-03T17:58:51Z', u'imkarthikk', False, u'User', 973265]
['IanLunn', 'Hover', u'2015-01-04T03:06:59Z', u'xiaokaike', False, u'User', 4251499]
['IanLunn', 'Hover', u'2015-01-04T06:39:23Z', u'jso0', False, u'User', 6308325]


In [7]:
mongocli.close()

In [8]:
def starring_etl(mongodb, coll_name, start_day):
    """ Description: function to flatten mongodb into dataframe
        - mongodb: mongodb database connection
        - coll_name: collection name
        - start_day: str, the day that the query starts from, ie. '2015-01-01'
    """
    stars_list = [] # init a list to contain query result
    stars = mongodb[coll_name].find({"starred_at": {"$gte": start_day}},
                                                    {"_id":0, "starred_at": 1, "user.type": 1, "user.id":1, "user.login":1, "user.site_admin":1})
    for star in stars:
        stars_list.append(coll_name.split('.')[1:3] + [star['starred_at']] + star['user'].values()) # append the flattened row
    # construct a dataframe
    stars_df = pd.DataFrame(stars_list, columns=['owner', 'repo', 'starred_at', 'user.login', 'user.site_admin', 'user.type', 'user.id'])
    print coll_name, stars_df.shape
    
    return stars_df

In [9]:
stars_df = starring_etl(mongodb, 'repos.IanLunn.Hover.stargazers', '2015-01-01')

repos.IanLunn.Hover.stargazers (9592, 7)


In [49]:
def dump_starring(db_url, db_name, start_day):
    """ Description: function to write 
        - db_url: mongodb url
        - db_name: mongodb database name
        - start_day: str, the day that the query starts from, ie. '2015-01-01'
    """
    mongocli = MongoClient(db_url)# init mongodb client
    mongodb = mongocli[db_name] # connect 'github' database
    coll_names = mongodb.collection_names() # get collection names
    
    for coll_name in coll_names: # loop collections
        # construct a dataframe
        stars_df = starring_etl(mongodb=mongodb, coll_name=coll_name, start_day=start_day)
        
        # write out csv
        file_name = 'starring_sample10.csv'
        if not os.path.isfile(file_name): # if file not exist, wirte column names
            stars_df.to_csv(file_name, index=False)
        else: # else it exists, not append colnames
            stars_df.to_csv(file_name, mode = 'a', header=False, index=False)
        print coll_name + 'is completed...'
        print '-------------------------------------------------'
        
    mongocli.close()

In [50]:
dump_starring(db_url, db_name, '2015-01-01')

repos.lebinh.ngxtop.stargazers (986, 7)
repos.lebinh.ngxtop.stargazersis completed...
-------------------------------------------------
repos.madebymany.sir-trevor-js.stargazers (866, 7)
repos.madebymany.sir-trevor-js.stargazersis completed...
-------------------------------------------------
repos.IanLunn.Hover.stargazers (9592, 7)
repos.IanLunn.Hover.stargazersis completed...
-------------------------------------------------
repos.jschr.bootstrap-modal.stargazers (877, 7)
repos.jschr.bootstrap-modal.stargazersis completed...
-------------------------------------------------
repos.eczarny.spectacle.stargazers (3814, 7)
repos.eczarny.spectacle.stargazersis completed...
-------------------------------------------------
repos.jquery.jquery-ui.stargazers (1455, 7)
repos.jquery.jquery-ui.stargazersis completed...
-------------------------------------------------
repos.JakeWharton.ViewPagerIndicator.stargazers (3547, 7)
repos.JakeWharton.ViewPagerIndicator.stargazersis completed...
--------

### starring

In [51]:
starrings = pd.read_csv('./starring_sample10.csv')

In [52]:
starrings.head()

,owner,repo,starred_at,user.login,user.site_admin,user.type,user.id
0,lebinh,ngxtop,2015-01-01T02:07:47Z,GameCracker,False,User,4250690
1,lebinh,ngxtop,2015-01-01T18:44:45Z,qorbani,False,User,771382
2,lebinh,ngxtop,2015-01-02T18:38:04Z,vrrs,False,User,2623289
3,lebinh,ngxtop,2015-01-02T21:35:50Z,Aldor007,False,User,2446168
4,lebinh,ngxtop,2015-01-03T12:45:05Z,azhurbilo,False,User,6387460


In [53]:
# transform isodate into datetime
starrings.starred_at = starrings.starred_at.apply(lambda x : parser.parse(x))

In [54]:
starrings['year'] = starrings.starred_at.dt.year
starrings['month'] = starrings.starred_at.dt.month
starrings['week'] = starrings.starred_at.dt.week
starrings['dayofweek'] = starrings.starred_at.dt.dayofweek

In [55]:
starrings.groupby(by=['owner', 'repo', 'year', 'month'])['starred_at'].count()

owner        repo                year  month
IanLunn      Hover               2015  1        2897
                                       2         847
                                       3         318
                                       4         339
                                       5         326
                                       6         290
                                       7         249
                                       8         254
                                       9         387
                                       10        357
                                       11        268
                                       12        307
                                 2016  1         395
                                       2         303
                                       3         427
                                       4         308
                                       5         319
                                       6         290
 

In [56]:
starrings.to_csv('starrings_group_sample10.csv', index=False)